In [49]:
import bs4
import urllib
import urllib.request
import pandas

In [50]:
#get donuts url
menu_url = "https://www.misterdonut.jp/m_menu/donut/"
req = urllib.request.Request(menu_url)
with urllib.request.urlopen(req) as res:
    menu_soup = bs4.BeautifulSoup(res.read(), "html.parser")

items = menu_soup.find_all('li', class_='mod_item clm')
items[:1]

[<li class="mod_item clm"><a href="/m_menu/donut/bake22_01.html">
 <div class="img"><figure><img alt="ベイク チーズタルトドーナツ" src="/m_menu/donut/images/144/product_1.jpg"/></figure></div>
 <div class="txt"><p>ベイク <br class="sp_only"/>チーズタルト<br/>ドーナツ</p></div>
 </a></li>]

In [51]:
donut_urls = ["https://www.misterdonut.jp"+item.find('a').get('href') for item in items]
donut_urls[:5]

['https://www.misterdonut.jp/m_menu/donut/bake22_01.html',
 'https://www.misterdonut.jp/m_menu/donut/bake22_02.html',
 'https://www.misterdonut.jp/m_menu/donut/bake22_06.html',
 'https://www.misterdonut.jp/m_menu/donut/bake22_07.html',
 'https://www.misterdonut.jp/m_menu/donut/bake22_08.html']

In [52]:
donut_url = donut_urls[0]
print(donut_url)
req = urllib.request.Request(donut_url)
with urllib.request.urlopen(req) as res:
    donut_soup = bs4.BeautifulSoup(res.read(), "html.parser")

https://www.misterdonut.jp/m_menu/donut/bake22_01.html


In [53]:
print(donut_soup.find('ul', class_="breadCrumb_wrap").find_all('li')[-1].text)
print(donut_soup.find('span', class_="price").text)
nutritions = donut_soup.find('div', class_="ingredient_list nutrition_list").find_all('li')
print(nutritions[0].find('span', class_="name").text)
print({nu.find('span', class_="name").text:nu.find('span', class_="add_comma").text for nu in nutritions})

ベイク チーズタルトドーナツ
230
熱量
{'熱量': '328', 'たんぱく質': '5.2', '脂質': '23.0', '炭水化物': '24.4', '食塩相当量': '0.7'}


In [54]:
allergys = donut_soup.find('div', class_="ingredient_list allergy_list").find_all('li')
print(allergys[0].find('span', class_="name").text, allergys[0].find('dd').text)

小麦 〇


In [55]:
soup_list = list()
for url in donut_urls:
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as res:
        soup = bs4.BeautifulSoup(res.read(), "html.parser")
    soup_list.append(soup)

In [56]:
dict_list = list()
for soup in soup_list:
    d = dict()
    d.update({"名前":soup.find('ul', class_="breadCrumb_wrap").find_all('li')[-1].text})
    d.update({"価格":soup.find('span', class_="price").text})
    nus = soup.find('div', class_="ingredient_list nutrition_list").find_all('li')

    d.update({nu.find('span', class_="name").text:nu.find('span', class_="add_comma").text for nu in nus})
    if int(d['熱量']) < 140:
        continue
    try:
        d.update({al.find('span', class_="name").text:al.find('dd').text for al in soup.find('div', class_="ingredient_list allergy_list").find_all('li')})
    except:
        print(d)
    dict_list.append(d)

{'名前': 'ふかふか焼きドーナッツ プレーン[ミニサイズ] 2個入り', '価格': '400', '熱量': '186', 'たんぱく質': '1.3', '脂質': '7.7', '炭水化物': '27.4', '食塩相当量': '0.4'}


In [57]:
df = pandas.DataFrame(dict_list)
df.fillna('－').to_csv('donut.csv')